<img src=../figures/Brown_logo.svg width=50%>

# Data-Driven Design & Analyses of Structures & Materials (3dasm)

## Lecture 26: A brief introduction to optimization (for ML)

### Suryanarayanan M. S. | <a href = "mailto: s.manojsanu@tudelft.nl">s.manojsanu@tudelft.nl</a>  | PhD Candidate

# Outline for today's lecture

<img align=right src=./figures/grr.png width=60%>

* Optimization problem formulation
    * Objective, variables and constraints
* Characteristics of optimization problems
    * Modality
    * Convexity
* Solving optimization problems
    * Classify optimizers
    * Taylor series expansion
    


**References:**
* *J. R. R. Martins* & Andrew Ning, Engineering Design Optimization, 2021 - Chapters 1 & 4
* For practical details of algorithms (Extra):
    * Nocedal, Jorge, and Stephen J. Wright. Numerical optimization. Springer Science & Business Media, 2006.

# Why optimization matters?

*In machine learning:*
1. **Parameter estimation**
    * Optimization is the tool to achieve our true goal: "generalization". 
    
    *  $\theta^* \in \text{arg}\,\min\limits_{\theta}\, \mathcal{L}(\mathbf{\theta})$, where $\mathcal{L}$ is the loss function and $\theta$ are the model parameters.
    

2. **Hyper-parameter tuning**
    * Anything can be a hyper-parameter!


*In the data-driven process:*

<img src=./figures/blocks.png width=100%>

# Setting up an optimization problem

**General formulation**

<img src=./figures/general_form.png width=50%>

* $\vec{x}$ are decision variables.
    * $\underline{x}$ and $\overline{x}$ are the lower and upper bounds.
* $f(\vec{x})$ is the objective function.
* $g$ and $h$ are constraint functions.

Note:
1. The lecture assumes this formulation
2. Different textbooks follow different conventions
    * Optimization packages (and sometimes functions within a package) might follow different formulaions
    * Always check whether the constraints are defined as given above (LHS form)
    * E.g. [Scipy](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#constrained-minimization-of-multivariate-scalar-functions-minimize) has two algorithms `SLSQP` and `trust-constr`. One uses a combined LHS and RHS form while the latter uses RHS form.

### A. Decision variables or Design variables

<img style="float: right;" src="./figures/dvs.png" width=30%>

* Represent the space of possibilities (aka Design space)
    * To be searched for the best solution
    
* Continuous or discrete

* Bounded or unbounded

* No: of design variables = dimensionaity

* Physically meaningful or abstract
    * E.g. thickness of a beam or a neural network's weight parameter

### B. Objective function

<img style="float: right;" src="./figures/objective.png" width=30%>

* Performance metric to be optimized

* Single or multiple objectives are possible

    * E.g. Cost vs quality, Weight vs stiffness etc.

* Can be linear or non-linear (in the design variables)

* E.g. weight of a beam or accuracy of a neural network

**Note**
- Optimization is often shown as a minimization problem. Maximization problems can be converted to minimization problems by multiplying the objective function by -1.

### C. Constraints

<img style="float: right;" src="./figures/constraint.png" width=40%>


* Functions that limit the design space.
    * Linear or non-linear
    * E.g. Stress in a beam should <= yield stress of the material.

* Makes life (and optimization) much more difficult.
    * Solution has to be **feasible** (not violate any constraints)
    * Solution has to be **optimal** (best possible solution)

* Equality or inequality
    * Inequality is more difficult to handle!
    * They can be on or off (active or inactive)
    
* Bounds on the design variables are called **box constraints**.
    * However, they are much easier to handle.

### Narrowing our focus

- Optimization is extremely vast
- Let's simplify things for ML
    - Discrete or continuous decision variables
    - Single objective
    - No constraints -> Unconstrained

<img align=center src="./figures/narrow_focus.png" width=50%>


# Characteristics of an optimization problem

* Attributes of the problem 
    * Helps classify the problem
    * May help in getting better solutions with less resources!
* Some important attributes are:
    * Smoothness of the functions involved
    * Linearity
    * Stochasticity
    * **Modality**
    * **Convexity**

### Modality

* Modes = Minima (Remember our convention!)
    * Minima are points that are better than their neighbours
    * $x^*$ is a minima if $f(x^*) \leq f(x)$ for every $x \in \{||x - x^*|| < \epsilon\ , \epsilon > 0 \}$
    * Global minima vs local minima
    
<img align=right src="./figures/modality.png" width=40%>

* Multi-modal functions cause problems (like multi-modal distributions)
    * Getting stuck in local minima
    * Every global minimum is also a local minimum
    
**How do we know we have found the best solution possible ?**
* In general, we don't!

### Convexity

* Purely mathematical
* **Golden rule: If an optimization problem is convex, all local minima are global minima**


* What is a convex optimization problem ?
    * If all functions involved are convex
    * ...

* Convex functions should meet 2 criteria

**A** : *The line segment joining any two points of the graph of the function has to be above the function itself.*
* For any two points $\vec{x}, \vec{y}$ in the domain of $f$ and variable $0 \leq \alpha \leq 1$ :
    
$$f(\alpha \vec{x} + (1 - \alpha) \vec{y}) \leq \alpha f(\vec{x}) + (1 - \alpha) f(\vec{y})$$
<img align=centre src="./figures/convexity.png" width=60%>

**B** : *The domain of the function should not have holes*
* The line segment joining two points of a set should also lie in the set
* For continuous design variables in unconstrained settings, this is always valid

<img align=centre src="./figures/convexity.png" width=60%>

# Solving optimization problems

* Searching the design space = optimizers
* Choice of optimizer is dependent on problem characteristics
    * No-free lunch theorem!

## Classification

<img align=centre src="./figures/optimizers.png" width=60%>

1. **Search strategy**
    * Local: Search in the viscinity of their initialization
        * Exploitative nature
        * Finds the nearest minima
        
    * Global: Search the entire design space (or try)
        * Exploratory in nature

2. **Algorithm design**
    * Heuristics
        * Nature inspired or based on rules-of-thumb
        * Robust (fewer assumptions) & general
    * Mathematically designed
        * Strictly converge to an optimal point*.
        * Works well when the assumptions match the problem
        * Very efficient
        * Difficult to design and implement

3. **Order of information used**

* Practically, the most important classification
* Lets dive a bit deeper!

## Taylor series and order of information


* Approximating a function $f$ at a given point $x_0$
    
$$f(x + \alpha)|_{x=x_0} \approx f(x_0) + \frac{df}{dx}\Big\lvert_{x=x0} \alpha + \frac{1}{2}\frac{d^2 f}{d x^2}\Big\lvert_{x=x_0}\alpha^2 + \mathcal{O}(\alpha^3)$$ 
 
 

        

* $\alpha$ is the perturbation
    * Since there is one direction, the perturbation is along $\hat{x}$
* More terms implies a better approximation
    * The error in the approximation scales as $\alpha^{n+1}$, if we include only n terms
        


**In n-dimensions**


$$ f(\vec{x} + \alpha \vec{p})|_{x=\vec{x}_0} \approx f(\vec{x}_0) + \alpha \nabla f(\vec{x}_0)^T \vec{p} + \frac{1} {2}\alpha^2 \vec{p}^T   \mathbf{H}(\vec{x}_0)   \vec{p}$$

* The main difference is that we perturb along a given direction!
* This is the form used for many optimization algorithms 

* Three terms in the series
    * The value of the function at the x (our point of interest) - **Zeroth term**
    
    * The gradient - vector of partial derivatives - **First order term**
        - Generalization of the first-derivative to n dimensions
        - Magnitude
         $$ ||\nabla f(\vec{x})|| = \Big[\frac{\partial f}{ \partial x_1}, \frac{\partial f}{ \partial x_2} ..., \frac{\partial f}{ \partial x_n} \Big]$$
        - Direction - steepest ascent of the function. i.e. the direction in which the function increases the fastest.

* Hessian -  matrix of second-order derivatives - **Second order term**
    - Generalization of the second-derivative to n dimensions - A symmetric matrix with size n x n
$$ H(\vec{x}) = \begin{bmatrix} \frac{\partial^2 f}{\partial x_1^2} & \cdots & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\ \vdots &  \ddots & \vdots \\ \frac{\partial^2 f}{\partial x_n \partial x_1} & \cdots & \frac{\partial^2 f}{\partial x_n^2} \end{bmatrix} $$

    - This matrix represents the curvature of the function (since it is a change in the gradients!).

* Zeroth-order optimizers
    * Uses only function evaluations
    * E.g. hyper-parameter tuning
* First-order optimizers
    * Gradient-based optimizers
    * Most common in ML
* Second-order optimizers
    * The fastest and most expensive optimizers
    * Not too popular in ML
    * E.g. K-FAC, Shampoo etc.

## Summary

* General form of an optimization problem
* Constraints make optimization difficult
* Convex optimization problems are super nice!
* Optimizers - Algorithms for searching the design space for solutions

**Next lecture**
* More about optimizers
* Optimality criteria - To identify minima
* Automatic differentiation